# 1. Keys and Signatures 

This is a short introduction to digital signatures by [Francesco Roda](mailto:francesco.roda@citi.com).  
Start from the top cell of the workbook and run each cell by pressing the play button at the top of the window ▶️

**The code**    
Below, we define the functions to generate a SECP256K1 key pair, sign a message with the private key and verify the digital signature with the public key.  
The code below uses two standard libraries, ***ecdsa*** and ***ecies***, which contain the definitions for the SECP256K1 cryptographic functions.  

#Cell 1

In [ ]:
print("Generating functions...") 

%pip install ecdsa -q
import ecdsa 
import binascii

def generate_keys():
    kpr = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    private_key = kpr.to_string().hex()
    kpu = kpr.get_verifying_key() 
    public_key = kpu.to_string().hex()
    return public_key, private_key #, kpr
def derive_public_key(private_key_asc):
    #kpr_bytes =codecs.decode(private_key_asc, 'hex') #back to bytes
    kpr_bytes = binascii.unhexlify(private_key_asc)
    kpu = ecdsa.SigningKey.from_string(kpr_bytes, curve=ecdsa.SECP256k1).verifying_key
    #kpu_hex = '04' + codecs.encode(kpu.to_string(), 'hex').decode("utf-8")  #back to string
    kpu_hex = '04' + binascii.b2a_hex(kpu.to_string()).decode("utf-8")
    return kpu_hex
def sign_message(message, private_key): 
    bmessage = message.encode()
    sk = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    signature = sk.sign(bmessage)
    return signature.hex(), message
def verify_signature(public_key, signature, message): 
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key), curve=ecdsa.SECP256k1)
    try: 
      vk.verify(bytes.fromhex(signature), message.encode())
      return True 
    except:
      return False
  

print("Functions generated, you can now use them on this workbook to generate key pairs, sign messages and verify digital signatures.") 

#Cell 2

**Create a key pair**  
We use the generate_key() function to randomly generate a private key and derive the corresponding public key.  
In this session we let the system generate a private key at random. In the next session we will explore interesting alternatives. 
The author (sender of the message) will use the private key to digitally sign a message. Readers will use the public key of the author to verify the integrity of the message published.  
The author will also be able to encrypt the message, using her private key.  
The author publishes his public key, the message and the digital signature. Of course the author does not publish the private key.  

#Cell 3

In [ ]:
# Here the keys are also assigned to variables.
public_key, private_key = generate_keys()
print("Public key:", public_key)
print("Private key:", private_key)

#Cell 4

**The sender uses his private key to sign a message**  
We use the sign_message() function to sign a message with the private key.

#Cell 5

In [ ]:
signature, message = sign_message("I owe John 21 pounds, Sincerely Francesco Roda", private_key)
print("Message: ", message)
print("Signature: ",signature)

#Cell 6

**The recipient uses the sender public key to verify the integrity of the message**  
We use the verify_signature() function to check the signature for a given message and a given private key.

#Cell 7

In [ ]:
test = verify_signature(public_key, '6b298bb3bc945d7ceec829c60779611f66f243a86fba766d0217702a7d60f2bedcc1958e7c5a5f25bb36dc929adf23d4830217f4045f43039d5b78aed2c0e016', message) #this is what miners do 
print("The digital signature proves the integrity of the message", test)

#Cell 8

**The recipient uses the sender public key to verify the integrity of another message with the same digital signature**  
We use the verify_signature() function to check the signature for a given message and a given private key.

#Cell 9

In [ ]:
message_changed = "I owe John 19 pounds, Sincerely Francesco Roda"
print("New message: ", message_changed)
test = verify_signature(public_key, signature, message_changed)
print("The digital signature proves the integrity of the message", test) 

#Cell 10

**"Sending" messages**  
Digitally signed messages can be send via email to the intended recipient but they do not guarantee privacy: if the message is intercepted, its content is clear for everyone to read.  
Public key cryptography can be used to enforce privacy of cimmunications.  
The recipe is as follows:  
> The recipient publishes her public key  
> The sender encrypts the message with her public key of the recipient  
> The encrypted message is published: there is no need to send it as no one but the recipient can decrypt it       
> The recipient decrypts the message with her private key to access its content   
>
💡 Public key cryptography is also called asymettric cryptography: the asymmetry refers to the role of the public key (encypt messages) and the need for the private key to decrypt them.
💡 More formally:   
> a single key is required to encypt a message (the public key of the recipient)  
> both keys are required to decrypt a message  
> the public key can be derived from the private key but **not vice versa**   
>
The code below defines functions that use the SECP256K1 algorithm to encrypt and decrypt messages.   

#Cell 11

In [ ]:
print("Generating functions...") 
%pip install eciespy -q
import binascii
from ecies.utils import generate_eth_key, hex2prv
from ecies import encrypt, decrypt
def encrypt_message(public_key, message):
    return encrypt(public_key, message.encode()).hex()
def decrypt_message(private_key, cipher):
    return decrypt(private_key,  bytes.fromhex(cipher)).decode()

print("Functions generated, you can now use them on this workbook to encrypt and decrypt messages.") 

#Cell 12

**Group Exercise**    
Work in pairs to complete the following tasks:  
> 1) Exchange a signed message and verify its integrity
> 2) Exchange an encrypted message
> 3) Exchange an encrypted and signed message  
>
💡 To complete these task each player would need to create a key pair. Players should not not disclose their private keys!  

💭 ***Use the functions defined in this workbook. For your convenience they are listed below***  
> generate_keys()  ℹ️ *Returns a public key and a private key. Store the private key somewhere you can retrieve it from and send the public key to anyone you wish to communicate with (ideally, you would publish it to a secure repository)*    
> derive_public_key(private_key_asc)  ℹ️ *Requires a valid private key. You can use the one that you generated and saved. You probably do not need this function unless you forget your public key (but you still have your private key)*       
> sign_message(message, private_key)  ℹ️ *Requires the message to be signed and the private key you will sign the message with. The function will return a signature, send it to anyone you wish to communicate with*  
> verify_signature(public_key, signature, message)  ℹ️ *Requires the publick key of the author of the message, the signature and the message to be verified*  
> encrypt_message(public_key, message)  ℹ️ *Requires the publick key of the recipient of the message, and the message to be encrypted. The funciton returns the encrypted (unreadaeble) message*      
> decrypt_message(private_key, encrypted message)  ℹ️ *Requires the private key of the recipient of the message, and the encrypted message to be decrypted*    
        
💭 ***Remember to use single or dourble quotes ('some text and 1 number' or "some text and 1 number") when entering strings of characters; alternatively use variables to store keys, signatures and messages, for example:***  
>    private_key, public_key = generate_keys()  
    *this function, unlike others, returns two values which you have to capture in two separate variables*  
    
#Cell 13

In [ ]:
#your code:

#Cell 14

**Conclusions**  
You have learned a key apsect of cryptography: digital signatures.  
💡 Blockchains use digital signatures to ensure that transactions are posted by the user that owns the account which funds are transferred from.  
Just like wet (traditional) signatures are used to ensure that a cheque is written by the owner of the account, digital signatures ensure that an account is operated only by its owner, the person who is in possession of the private key.  
Blockchain transactions are like cheques: as soon as they are signed, they can be posted to the blockchain for the funds be transferred to the intended recipient.  
Wet signatures can be counterfaited - digital signatures, that rely on strong cryptography, cannot!     

#Cell 15